<a href="https://colab.research.google.com/github/negarhonarvar/Persian-NLP/blob/main/Persiantokenization%26morphologicalanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this project, we aim to extract the key linguistic units in Persian that carry semantic importance. More specifically, it includes:

    

1.   Important Words: The content-bearing words in the text.
2.   Morphological Components: This means every significant affix (both prefixes and suffixes), the root forms of verbs, as well as separate conjunctions and nouns.

3.   Preservation of Punctuation and Numbers: These are treated as effective units too.


The concept is closely related to tokenization and morphological analysis and our goal is to break down the target text into its smallest meaningful parts without redundancy and implement a reusable algorithm.

### Libraries

In [1]:
!pip install PyPDF2

In [2]:
!pip install pypdf
!pip install arabic-reshaper python-bidi

In [3]:
!pip install --upgrade PyMuPDF

In [4]:
!pip install fitz

In [5]:
!pip install tools

In [6]:
!pip install parsivar

In [7]:
!pip install hazm

In [8]:
!pip uninstall fitz
!pip install --upgrade --force-reinstall pymupdf

Found existing installation: fitz 0.0.1.dev2
Uninstalling fitz-0.0.1.dev2:
  Would remove:
    /usr/local/bin/fitz
    /usr/local/bin/log2design.py
    /usr/local/lib/python3.11/dist-packages/.DS_Store
    /usr/local/lib/python3.11/dist-packages/fitz-0.0.1.dev2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/fitz/*
    /usr/local/lib/python3.11/dist-packages/scripts/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.11/dist-packages/fitz/table.py
    /usr/local/lib/python3.11/dist-packages/fitz/utils.py
    /usr/local/lib/python3.11/dist-packages/scripts/readme-gen/readme_gen.py
Proceed (Y/n)? غ
Your response ('غ') was not one of the expected responses: y, n, 
Proceed (Y/n)? y
y
  Successfully uninstalled fitz-0.0.1.dev2
  Using cached pymupdf-1.25.3-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (3.4 kB)
Using cached pymupdf-1.25.3-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (20.0 MB)
  Attempting uninstall: pymupdf
    F

In [9]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.0 MB/s eta 0:00:00


In [10]:
!apt-get install -y poppler-utils tesseract-ocr tesseract-ocr-fas
!pip install pdf2image pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-fas tesseract-ocr-osd
0 upgraded, 5 newly installed, 0 to remove and 29 not upgraded.
Need to get 5,303 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 te

In [11]:
import PyPDF2
import unicodedata
import pandas as pd
from pypdf import PdfReader
import arabic_reshaper
import re
from bidi.algorithm import get_display
import pdfplumber
import pytesseract
from pdf2image import convert_from_path

# Section 1: Extract the Text

### Text Extraction from PDF

In [12]:
from google.colab import drive
drive.mount('/content/drive')

pdf_path  = "/content/drive/MyDrive/NLP/edit.pdf"

Mounted at /content/drive


In [13]:
df = pd.DataFrame(columns=["text"])
df

,text


The target file encoding was causing troubles in extracting the data in the correct format. Since I didn't wanted to change anything from the initial file, I used OCR methods to correctly extract the data from the pdf file by taking advantage of Tesseract.



In [14]:
def extract_text(pdf_path, start_page=0, end_page=None, dpi=300):
    pages = convert_from_path(pdf_path, dpi=dpi)
    if end_page is None or end_page > len(pages):
        end_page = len(pages)

    extracted_text = ""
    for i in range(start_page, end_page):
        page_image = pages[i]
        # Use Tesseract to extract text from the image specifying Persian language ("fas")
        page_text = pytesseract.image_to_string(page_image, lang='fas')
        extracted_text += page_text + "\n"
    return extracted_text


### Text Normalization

In [15]:
def normalize_text(text):
    normalized = unicodedata.normalize('NFKC', text)
    normalized = " ".join(normalized.split())
    return normalized

In [16]:
def reshape_bidi_text(text):
    def contains_presentation_forms(s):
        for c in s:
            if '\uFB50' <= c <= '\uFDFF' or '\uFE70' <= c <= '\uFEFF':
                return True
        return False

    if contains_presentation_forms(text):
        return text
    else:
        config = arabic_reshaper.Config(delete_harakat=False, support_ligatures=True)
        reshaped_text = arabic_reshaper.reshape(text, config=config)
        bidi_text = get_display(reshaped_text)
        return bidi_text

In [17]:
raw_text = extract_text(pdf_path, start_page=0, end_page=8)
raw_text

'با عرض سلام و خسته نباشید\n\nلطفاً از متن پیوست\u200cشده, کمترین تعداد واحد موثر را بدون حذف واحد موثر دیگری استخراج کرده و در\nیک فایل 05۷ ذخیره کنید.\n\nتوضیحات و شرایط:\n\nشما مجاز به استفاده از تمامی کتابخانه\u200cها و مدل\u200cهای متن\u200cباز برای انجام این پروژه هستید.\nالگوریتم باید کاملا تکرارپذیر و منسجم باشد. از هرگونه کاردستی خودداری شود.\n\nواحد موثر فارسی برای ما اهمیت بیشتری دارند؛ کلمات انگلیسی را می\u200cتوانید حذف کنید.\nعلائم نگارشی و اعداد باید حفظ شوند و به\u200cعنوان واحد موثر لحاظ گردند.\n\nهدف کمینه کردن تعداد واحدهای موثر با حفظ تمامی واحدهای موثر است.\n\nدا نخ دن چا س\n\nنکته:\n\nتن مورد نظر:\n\n"به گزارش خبرآنلاین به نقل از ایسنا وزارت دفاع روسیه روز پنج\u200cشنبه اعلام کرد\nنیروهای مسلح روسیه در واکنش به حملات کی\u200cبیف در عمق خاک روسیه, مکان\u200cهای\nسامانه\u200cهای تسلیحاتی دوربرد اوکراین را در طول هفته مورد حمله قرار داده\u200cاند. در\nواکنش به این تحولات کاخ سفید در بیانیه\u200cای به نقل از بایدن گفت: «اين حمله\nوحشتناک و عصبانی\u200cکننده است و یادآ

In [18]:
normalized_text = normalize_text(raw_text)
normalized_text

'با عرض سلام و خسته نباشید لطفاً از متن پیوست\u200cشده, کمترین تعداد واحد موثر را بدون حذف واحد موثر دیگری استخراج کرده و در یک فایل 05۷ ذخیره کنید. توضیحات و شرایط: شما مجاز به استفاده از تمامی کتابخانه\u200cها و مدل\u200cهای متن\u200cباز برای انجام این پروژه هستید. الگوریتم باید کاملا تکرارپذیر و منسجم باشد. از هرگونه کاردستی خودداری شود. واحد موثر فارسی برای ما اهمیت بیشتری دارند؛ کلمات انگلیسی را می\u200cتوانید حذف کنید. علائم نگارشی و اعداد باید حفظ شوند و به\u200cعنوان واحد موثر لحاظ گردند. هدف کمینه کردن تعداد واحدهای موثر با حفظ تمامی واحدهای موثر است. دا نخ دن چا س نکته: تن مورد نظر: "به گزارش خبرآنلاین به نقل از ایسنا وزارت دفاع روسیه روز پنج\u200cشنبه اعلام کرد نیروهای مسلح روسیه در واکنش به حملات کی\u200cبیف در عمق خاک روسیه, مکان\u200cهای سامانه\u200cهای تسلیحاتی دوربرد اوکراین را در طول هفته مورد حمله قرار داده\u200cاند. در واکنش به این تحولات کاخ سفید در بیانیه\u200cای به نقل از بایدن گفت: «اين حمله وحشتناک و عصبانی\u200cکننده است و یادآوری دیگری از فوریت و آهمیت حمایت ا

# Section 2: Preprocessing

In [19]:
def remove_english_words(text):
    tokens = text.split()
    filtered_tokens = [token for token in tokens if not re.search(r'[a-zA-Z]', token)]
    return " ".join(filtered_tokens)

In [20]:
filtered_text = remove_english_words(normalized_text)
matches = re.findall(r'"(.*?)"', filtered_text, re.DOTALL)
for i, match in enumerate(matches, 1):
        cleaned_match = match.strip()
        print(f"String {i}: {cleaned_match}")
        df.loc[len(df)] = cleaned_match
print(df.head())

String 1: به گزارش خبرآنلاین به نقل از ایسنا وزارت دفاع روسیه روز پنج‌شنبه اعلام کرد نیروهای مسلح روسیه در واکنش به حملات کی‌بیف در عمق خاک روسیه, مکان‌های سامانه‌های تسلیحاتی دوربرد اوکراین را در طول هفته مورد حمله قرار داده‌اند. در واکنش به این تحولات کاخ سفید در بیانیه‌ای به نقل از بایدن گفت: «اين حمله وحشتناک و عصبانی‌کننده است و یادآوری دیگری از فوریت و آهمیت حمایت از مردم اوکراین در دفاع از آنها است. کی رئیس‌جمهور امریکا همچنین بار دیگر اعلام کرد واشنگتن به ارائه کمک‌های نظامی به اوکراین ادامه خواهد داد و افزود که محموله بعدی «در دست انجام است. ک به گزارش نووستی, , گزارش‌ها حاکی از آن است که دولت بایدن در حال اماده‌سازی بسته کمکی ۷۲۵ میلیون دلاری برای اوکراین است. انتظار می‌رود این بسته شامل انواع سلاح‌های ضد تانک از انبارهای اپالات متحجده؛ از جمله مین‌های زمینی, پهپادها, موشک‌های استینگر و مهمات برای سامانه‌های موشکی توپخانه با تحرک بالا (هیمارس) باشد. «پت رایدر» سخنگوی پنتااگون تأّیید کرد که دولت بایدن به کی‌یف اجازه داده است که از موشک‌های تدارکاتی دوربرد آمریکا مانند اتکمز بر

# Section 3: Tokenization & Morphological Analysis

In [21]:
from hazm import Normalizer as HazmNormalizer, word_tokenize, Stemmer, Lemmatizer

In [22]:
def morphological_segmentation(text):
    hazm_normalizer = HazmNormalizer()
    normalized_text = hazm_normalizer.normalize(text)
    tokens = word_tokenize(normalized_text)
    stemmer = Stemmer()
    lemmatizer = Lemmatizer()
    results = []
    for token in tokens:
        token_stem = stemmer.stem(token)
        token_lemma = lemmatizer.lemmatize(token)
        results.append((token, token_stem, token_lemma))
    return results

In [23]:
df_morph = pd.DataFrame(columns=["segment", "token", "stem", "lemma"])

I used Parsivar Stemmer to attain better precision in reverting each verb to its simple form.

In [25]:
import re
from parsivar import FindStems

stemmer = FindStems()

def simplify_verb(word):
    if not isinstance(word, str):
        return word

    # remove spaces, zero-width joiners, and thin spaces (U+2009)
    cleaned_word = word.replace(" ", "").replace("\u200c", "").replace("\u2009", "")

    if re.fullmatch(r'^بوده?(یم|ید|ند|ست|اند|باشند)?$', cleaned_word):
        return "بود"

    if re.fullmatch(r'^هست(یم|ید|ند|م|ی|یند)?$', cleaned_word):
        return "است"

    stem_result = stemmer.convert_to_stem(cleaned_word)

    """
     some parts of my data are shown this way after stemming:
     شست#شوی
     the stemmizer is inverting each verb into its root by adding the root in front of it after a #, so,
     instead of simply removing #, we should remove the # and anything which is placed before the # in the string.
    """

    if '#' in stem_result:
        parts = stem_result.split('#')
        result = parts[-1]
    elif '&' in stem_result:
        parts = stem_result.split('&')
        result = parts[-1]
    else:
        result = stem_result

    # replacing any residual thin space with a normal space and collapse extra whitespace.

    result = result.replace("\u2009", " ")
    result = " ".join(result.split())

    return result

In [26]:
def normalize_persian_word(word):

    word = word.replace("#", " ").replace("&", " ")
    word = word.replace("\u200c", "")
    word = " ".join(word.split())
    if re.fullmatch(r'^هست(یم|ید|ند|م|ی|یند)?$', word):
        return "است"
    suffixes = ["هایم", "های", "ها", "اش", "ات", "ایم", "ای", "ید", "ند", "م", "ی"]
    for suf in suffixes:
        if word.endswith(suf):
            candidate = word[:-len(suf)]
            if len(candidate) >= 2:
                word = candidate
                break
    return word

In [27]:
for idx, row in df.iterrows():
    segment = row["text"]
    morph_results = morphological_segmentation(segment)
    for token, stem, lemma in morph_results:
        df_morph.loc[len(df_morph)] = [segment, token, stem, lemma]

print("Extracted Segments:")
print(df.head())

Extracted Segments:
                                                text
0  به گزارش خبرآنلاین به نقل از ایسنا وزارت دفاع ...
1                                                   
2  بنابراین, ما نیز آماده‌ایم تا در صورت انجام اق...
3  این سریال خیلی قشنگه حتی از سریال شهرزاد هم قش...
4                    مررررررررسی عالی عالی عالی عالی


In [28]:
df_stemmed = pd.DataFrame(columns=["verb_simplified"])

In [29]:
df_stemmed["verb_simplified"] = df_morph["token"].apply(lambda x: simplify_verb(x) if isinstance(x, str) else x)

In [30]:
df_morph

,segment,token,stem,lemma
0,به گزارش خبرآنلاین به نقل از ایسنا وزارت دفاع ...,به,به,به
1,به گزارش خبرآنلاین به نقل از ایسنا وزارت دفاع ...,گزارش,گزار,گزارش
2,به گزارش خبرآنلاین به نقل از ایسنا وزارت دفاع ...,خبرآنلاین,خبرآنلاین,خبرآنلاین
3,به گزارش خبرآنلاین به نقل از ایسنا وزارت دفاع ...,به,به,به
4,به گزارش خبرآنلاین به نقل از ایسنا وزارت دفاع ...,نقل,نقل,نقل
...,...,...,...,...
599,عجیبه این سریال دیپلمات نیست هنرپيشه ها اشخاص ...,نیست,نیس,نیست
600,عجیبه این سریال دیپلمات نیست هنرپيشه ها اشخاص ...,هنرپیشه‌ها,هنرپیشه,هنرپیشه
601,عجیبه این سریال دیپلمات نیست هنرپيشه ها اشخاص ...,اشخاص,اشخاص,اشخاص
602,عجیبه این سریال دیپلمات نیست هنرپيشه ها اشخاص ...,دیگری,دیگر,دیگری


In [31]:
df_stemmed

,verb_simplified
0,به
1,گزارش
2,خبرآنلاین
3,به
4,نقل
...,...
599,نیست
600,هنرپیشه
601,اشخاص
602,دیگری


# Section 4: Minimal Set Extraction

In [40]:
def clean_effective_units(effective_units):
    valid_short_words = {"از", "به", "در", "با", "بر", "تا", "که", "را", "یا", "این", "آن", "بیا", "چون"}

    cleaned_units = set()
    for token in effective_units:

        if '#' in token:
            token = token.split('#')[-1]
        elif '&' in token:
            token = token.split('&')[-1]

        token_clean = (token.replace(" ", "")
                             .replace("\u200c", "")
                             .replace("\u200e", "")
                             .replace("\u200f", "")
                             .replace("\u2009", " "))

        token_clean = " ".join(token_clean.split())

        if re.search(r'[a-zA-Z]', token_clean):
            continue

        if len(token_clean) < 3 and token_clean not in valid_short_words:
            continue

        cleaned_units.add(token_clean)

    final_units = set()
    for token in cleaned_units:
        is_fragment = any(token != other and token in other for other in cleaned_units)
        if not is_fragment:
            final_units.add(token)

    return final_units

we aim to extract the minimal set of effective units from the morphologically segmented data stored in (df_morph)

In [32]:
df_morph["verb_simplified"] = df_stemmed["verb_simplified"].values

In [36]:
effective_units = set()

for idx, row in df_morph.iterrows():
    token = row["token"].strip()
    stem = row["stem"].strip()
    lemma = row["lemma"].strip()
    verb_simplified = row["verb_simplified"].strip()
    if token:
        effective_units.add(token)
    if stem:
        effective_units.add(stem)
    if lemma:
        effective_units.add(lemma)
    if verb_simplified:
        effective_units.add(verb_simplified)

In [37]:
effective_units

{'#هست',
 '(',
 ')',
 ',',
 '.',
 ':',
 '«',
 '»',
 '؛',
 '؟',
 'آزما',
 'آزمایش',
 'آماد#آمای',
 'آماده',
 'آمادهایم',
 'آماده\u200cایم',
 'آمریکا',
 'آمریکایی',
 'آموزش',
 'آموزشی',
 'آن',
 'آنها',
 'آهم',
 'آهمیت',
 'اتکمز',
 'اجازه',
 'اختلاف',
 'ادامه',
 'ادما',
 'ارائه',
 'ارز',
 'ارزش',
 'از',
 'اس',
 'است',
 'استفاده',
 'استینگر',
 'اشخاص',
 'اصلا',
 'اعلا',
 'اعلام',
 'افزا',
 'افزایش',
 'افزود',
 'افزود#افزا',
 'اقدا',
 'اقدامات',
 'امادهسازی',
 'اماده\u200cساز',
 'اماده\u200cسازی',
 'امریکا',
 'انبار',
 'انبارهای',
 'انتظار',
 'انجا',
 'انجام',
 'انواع',
 'انگلیس',
 'انگلیسی',
 'اهداف',
 'اهدافی',
 'اهم',
 'اهمیت',
 'او',
 'اورشنیک',
 'اول',
 'اولش',
 'اون',
 'اوکراین',
 'اپ',
 'اپال',
 'اپالات',
 'اگه',
 'ایران',
 'ایرانی',
 'ایسنا',
 'این',
 'اینکه',
 'ب',
 'با',
 'باب',
 'بابت',
 'بار',
 'باز',
 'بازی',
 'باشد',
 'باشید#باش',
 'بالا',
 'بالستیک',
 'بایدن',
 'ببینید',
 'بدرد',
 'بدید',
 'بر',
 'برا',
 'برای',
 'برمیگرد',
 'برمیگردیم',
 'بریانسک',
 'بزارید',
 'بزارید,',
 'ب

In [41]:
cleaned_units = clean_effective_units(effective_units)
minimal_units = sorted(cleaned_units)
df_minimal = pd.DataFrame(minimal_units, columns=["effective_unit"])
df_minimal

,effective_unit
0,آزمایش
1,آمادهایم
2,آمای
3,آمریکایی
4,آموزشی
...,...
264,یادمون
265,یگید
266,۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱۱...
267,۳۲۵۲۲


In [42]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df_minimal)

https://docs.google.com/spreadsheets/d/1ePQuIpICCtUdNNrrIO1Him8tfmTtROSWes83ScdJm9c#gid=0
